In [1]:
import requests
import pandas as pd
import sqlite3
from secrets import token

In [2]:
params = {
  'access_key': token,
  'dep_iata': ["BIO"]
}

api_result = requests.get('http://api.aviationstack.com/v1/flights', params)
api_response = api_result.json()

In [4]:
#api_response

In [23]:
flight_lst = []

for flight in api_response["data"]:

    flight_dict = {}

    flight_dict['flight_date'] = flight['flight_date']
    flight_dict['departure_airport'] = flight['departure']['airport']
    flight_dict['departure_terminal'] = flight['departure']['terminal']
    flight_dict['departure_gate'] = flight['departure']['gate']
    flight_dict['departure_delay'] = flight['departure']['delay']
    flight_dict['departure_scheduled'] = flight['departure']['scheduled']
    flight_dict['departure_estimated'] = flight['departure']['estimated']
    flight_dict['arrival_airport'] = flight['arrival']['airport']
    flight_dict['arrival_terminal'] = flight['arrival']['terminal']
    flight_dict['arrival_gate'] = flight['arrival']['gate']
    flight_dict['arrival_delay'] = flight['arrival']['delay']
    flight_dict['arrival_scheduled'] = flight['arrival']['scheduled']
    flight_dict['arrival_estimated'] = flight['arrival']['estimated']
    flight_dict['airline_name'] = flight['airline']['name']
    flight_dict['flight_number'] = flight['flight']['number']

    flight_lst.append(flight_dict)

In [44]:
flights_df = pd.DataFrame(
    flight_lst, 
    columns = ['flight_date', 'departure_airport', 'departure_terminal', 
        'departure_gate', 'departure_delay', 'departure_scheduled', 
        'departure_estimated', 'arrival_airport', 'arrival_terminal', 
        'arrival_gate', 'arrival_delay', 'arrival_scheduled', 'arrival_estimated', 
        'airline_name', 'flight_number'])
flights_df["flight_date"] = pd.to_datetime(flights_df["flight_date"])
flights_df["departure_scheduled"] = pd.to_datetime(flights_df["departure_scheduled"])
flights_df["departure_estimated"] = pd.to_datetime(flights_df["departure_estimated"])
flights_df["arrival_scheduled"] = pd.to_datetime(flights_df["arrival_scheduled"])
flights_df["arrival_estimated"] = pd.to_datetime(flights_df["arrival_estimated"])

In [34]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   flight_date          100 non-null    datetime64[ns]     
 1   departure_airport    100 non-null    object             
 2   departure_terminal   2 non-null      object             
 3   departure_gate       44 non-null     object             
 4   departure_delay      50 non-null     float64            
 5   departure_scheduled  100 non-null    datetime64[ns, UTC]
 6   departure_estimated  100 non-null    datetime64[ns, UTC]
 7   arrival_airport      100 non-null    object             
 8   arrival_terminal     90 non-null     object             
 9   arrival_gate         7 non-null      object             
 10  arrival_delay        4 non-null      float64            
 11  arrival_scheduled    100 non-null    datetime64[ns, UTC]
 12  arrival_estimated    10

In [54]:
#print(flights_df.head(5))

In [57]:
conn = sqlite3.connect("flights.sqlite3")
c = conn.cursor()

In [52]:
create_flights_query = """
    CREATE TABLE IF NOT EXISTS flight (
        [index] TEXT,
        [flight_date] TEXT,
        [departure_airport] TEXT,
        [departure_terminal] TEXT,
        [departure_gate] TEXT,
        [departure_delay] REAL,
        [departure_scheduled] TEXT,
        [departure_estimated] TEXT,
        [arrival_airport] TEXT,
        [arrival_terminal] TEXT,
        [arrival_gate] TEXT,
        [arrival_delay] REAL,
        [arrival_scheduled] TEXT,
        [arrival_estimated] TEXT,
        [airline_name] TEXT,
        [flight_number] TEXT        
    );
"""

c.execute(create_flights_query)           
conn.commit()

In [42]:
#flights_df = pd.read_sql_query("SELECT * FROM flight", conn)

In [45]:
#flights_df.info()

In [53]:
flights_df.to_sql('flight', con = conn, if_exists = 'append')

100

In [60]:
#resp_flights_df = pd.read_sql_query("SELECT * FROM flight", conn)

In [61]:
#resp_flights_df.head()